In [2]:
import numpy as np
import sklearn
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics 
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingRegressor

In [3]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

train=train[train.y<250]

In [4]:
train['Z0'] = train.groupby('X0')['y'].transform('mean')
train['Z1'] = train.groupby('X1')['y'].transform('mean')
train['Z2'] = train.groupby('X2')['y'].transform('mean')
train['Z3'] = train.groupby('X3')['y'].transform('mean')
train['Z4'] = train.groupby('X4')['y'].transform('mean')
train['Z5'] = train.groupby('X5')['y'].transform('mean')
train['Z6'] = train.groupby('X6')['y'].transform('mean')
train['Z8'] = train.groupby('X8')['y'].transform('mean')

In [5]:
test['Z0']=test['X0'].map(train.groupby('X0')['y'].mean())
test['Z1']=test['X1'].map(train.groupby('X1')['y'].mean())
test['Z2']=test['X2'].map(train.groupby('X2')['y'].mean())
test['Z3']=test['X3'].map(train.groupby('X3')['y'].mean())
test['Z4']=test['X4'].map(train.groupby('X4')['y'].mean())
test['Z5']=test['X5'].map(train.groupby('X5')['y'].mean())
test['Z6']=test['X6'].map(train.groupby('X6')['y'].mean())
test['Z8']=test['X8'].map(train.groupby('X8')['y'].mean())

In [6]:
train = train.sort_values(by='ID')

med = train['y'].mean()
print(med)

test['Z0']=test['Z0'].fillna(med)
test['Z1']=test['Z1'].fillna(med)
test['Z2']=test['Z2'].fillna(med)
test['Z3']=test['Z3'].fillna(med)
test['Z4']=test['Z4'].fillna(med)
test['Z5']=test['Z5'].fillna(med)
test['Z6']=test['Z6'].fillna(med)
test['Z8']=test['Z8'].fillna(med)

100.630190114


In [7]:
for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder() 
        lbl.fit(list(train[c].values) + list(test[c].values)) 
        train[c] = lbl.transform(list(train[c].values))
        test[c] = lbl.transform(list(test[c].values))



# shape        
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

from sklearn.decomposition import PCA, FastICA
n_comp = 10

# PCA
pca = PCA(n_components=n_comp, random_state=42)
pca2_results_train = pca.fit_transform(train.drop(["y"], axis=1))
pca2_results_test = pca.transform(test)

# ICA

ica = FastICA(n_components=n_comp, random_state=42)
ica2_results_train = ica.fit_transform(train.drop(["y"], axis=1))
ica2_results_test = ica.transform(test)

# Append decomposition components to datasets
for i in range(1, n_comp+1):
    train['pca_' + str(i)] = pca2_results_train[:,i-2]
    test['pca_' + str(i)] = pca2_results_test[:, i-2]
    
    train['ica_' + str(i)] = ica2_results_train[:,i-2]
    test['ica_' + str(i)] = ica2_results_test[:, i-2]
    
print('Shape train: {}\nShape test: {}'.format(train.shape, test.shape))

Shape train: (4208, 386)
Shape test: (4209, 385)
Shape train: (4208, 406)
Shape test: (4209, 405)


In [8]:
y = np.array(train["y"])
X=np.array(train.drop('y', axis=1))
print('Shape X: {}\nShape ty: {}'.format(X.shape, y.shape))
print 'Split sucessful'

Shape X: (4208, 405)
Shape ty: (4208,)
Split sucessful


In [ ]:
print 'GBR'

clf = GradientBoostingRegressor(n_estimators=500, max_depth=4, learning_rate=0.05)
np.mean(cross_val_score(clf, X, y, verbose=0, n_jobs=-1, cv=5))

GBR


In [ ]:
test['y']=y0
test

In [ ]:
output=test[['ID', 'y']]
output.to_csv('output.csv', index=False)